In [1]:
%tensorflow_version 2.x

In [2]:
import sys
import os

import matplotlib.pyplot as plt
import numpy as np
import skimage.io as io

import tensorflow as tf
import tensorflow_datasets as tfds

%matplotlib inline

#### Загрузка данных
Создадим папку для данных

In [3]:
!mkdir -p data

Загружаем данные

In [4]:
!cd data && wget http://images.cocodataset.org/zips/train2017.zip
!cd data && wget http://images.cocodataset.org/zips/val2017.zip 
!cd data && wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip


--2021-10-14 18:20:50--  http://images.cocodataset.org/zips/train2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 52.217.198.137
Connecting to images.cocodataset.org (images.cocodataset.org)|52.217.198.137|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19336861798 (18G) [application/zip]
Saving to: ‘train2017.zip.1’

train2017.zip.1       1%[                    ] 219.81M  67.5MB/s    eta 4m 48s ^C
--2021-10-14 18:20:54--  http://images.cocodataset.org/zips/val2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 52.217.198.137
Connecting to images.cocodataset.org (images.cocodataset.org)|52.217.198.137|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 815585330 (778M) [application/zip]
Saving to: ‘val2017.zip.1’

val2017.zip.1       100%[===================>] 777.80M  77.3MB/s    in 10s     

2021-10-14 18:21:04 (75.6 MB/s) - ‘val2017.zip.1’ saved [815585330/815585330]

--2021-10-14 18:21:04--  http:/

Распаковка данных

In [5]:
!cd data && unzip -q train2017.zip
!cd data && unzip -q val2017.zip
!cd data && unzip -q annotations_trainval2017.zip

replace train2017/000000147328.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace train2017/000000414738.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: N
replace val2017/000000212226.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: NA
replace annotations/instances_train2017.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: NA


#### COCO API

In [6]:
!cd data && git clone https://github.com/cocodataset/cocoapi
!cd data/cocoapi/PythonAPI && make

fatal: destination path 'cocoapi' already exists and is not an empty directory.
python setup.py build_ext --inplace
running build_ext
skipping 'pycocotools/_mask.c' Cython extension (up-to-date)
building 'pycocotools._mask' extension
creating build
creating build/common
creating build/temp.linux-x86_64-3.7
creating build/temp.linux-x86_64-3.7/pycocotools
x86_64-linux-gnu-gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fdebug-prefix-map=/build/python3.7-Y7dWVB/python3.7-3.7.12=. -fstack-protector-strong -Wformat -Werror=format-security -g -fdebug-prefix-map=/build/python3.7-Y7dWVB/python3.7-3.7.12=. -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/local/lib/python3.7/dist-packages/numpy/core/include -I../common -I/usr/include/python3.7m -c ../common/maskApi.c -o build/temp.linux-x86_64-3.7/../common/maskApi.o -Wno-cpp -Wno-unused-function -std=c99
../common/maskApi.c: In function ‘rleDecode’:
../commo

In [7]:
from pycocotools.coco import COCO
COCO_ROOT = './data/'

sys.path.insert(0, os.path.join(COCO_ROOT, 'cocoapi/PythonAPI'))

Универсальный датасет для классификации

In [8]:
class Dataset():

    def crop_images(self, img, inp_size, random_crop=False):
        shape = tf.shape(img)
        pad = (
            [0, tf.maximum(inp_size - shape[0], 0)],
            [0, tf.maximum(inp_size - shape[1], 0)],
            [0, 0],
        )
        img = tf.pad(img, pad)

        if random_crop:
            img = tf.image.random_crop(img, (inp_size, inp_size, shape[2]))
        else: # central crop
            shape = tf.shape(img)
            ho = (shape[0] - inp_size) // 2
            wo = (shape[1] - inp_size) // 2
            img = img[ho:ho+inp_size, wo:wo+inp_size, :]

        return img

    def train_dataset(self, batch_size, epochs, inp_size):

        def item_to_images(item):
            random_crop = True
            img_combined = tf.py_function(self.read_images, [item], tf.uint8)
            img_combined = self.crop_images(img_combined, inp_size, random_crop)

            img = tf.cast(img_combined[...,:3], tf.float32) / np.float32(255.)
            mask_class = tf.cast(img_combined[...,3:4], tf.float32)
            return img, mask_class

        dataset = tf.data.Dataset.from_tensor_slices(self.img_list)
        dataset = dataset.shuffle(buffer_size=len(self.img_list))
        dataset = dataset.map(item_to_images)
        dataset = dataset.repeat(epochs)
        dataset = dataset.batch(batch_size, drop_remainder=True)

        return dataset

    def val_dataset(self, batch_size, inp_size):

        def item_to_images(item):
            random_crop = False
            img_combined = tf.py_function(self.read_images, [item], tf.uint8)
            img_combined = self.crop_images(img_combined, inp_size, random_crop)

            img = tf.cast(img_combined[...,:3], tf.float32) / np.float32(255.)
            mask_class = tf.cast(img_combined[...,3:4], tf.float32)
            return img, mask_class

        dataset = tf.data.Dataset.from_tensor_slices(self.img_list)
        dataset = dataset.map(item_to_images)
        dataset = dataset.batch(batch_size, drop_remainder=True)

        return dataset

#### Класс для сегментационного датасета COCO
Наследутся от универсльного Dataset и реализует кастомную функцию чтения данных.

In [9]:
class COCO_Dataset(Dataset):

    def __init__(self, sublist):
        ann_file_fpath = os.path.join(COCO_ROOT, 'annotations', 'instances_'+sublist+'2017.json')
        self.coco = COCO(ann_file_fpath)
        self.cat_ids = self.coco.getCatIds(catNms=['person'])
        # Загружается не весь сет, а 4000
        temp_img_list = self.coco.getImgIds(catIds=self.cat_ids)
        self.img_list = []
        for img_id in temp_img_list:
          if len(self.img_list) < 4000:
            img_data = self.coco.loadImgs(img_id)[0]
            img_fname = '/'.join(img_data['coco_url'].split('/')[-2:])
            if os.path.isfile(os.path.join(COCO_ROOT, img_fname)):
              self.img_list.append(img_id)
          else:
            break

    def read_images(self, img_id):
        img_id = int(img_id.numpy())
        img_data = self.coco.loadImgs(img_id)[0]
        img_fname = '/'.join(img_data['coco_url'].split('/')[-2:])

        img = io.imread(os.path.join(COCO_ROOT, img_fname))
        if len(img.shape) == 2:
            img = np.tile(img[..., None], (1, 1, 3))

        ann_ids = self.coco.getAnnIds(imgIds=img_data['id'], catIds=self.cat_ids, iscrowd=None)
        anns = self.coco.loadAnns(ann_ids)
        mask_class = np.zeros((img.shape[0], img.shape[1]), dtype=np.uint8)
        for i in range(len(anns)):
            mask_class += self.coco.annToMask(anns[i])
        mask_class = (mask_class > 0).astype(np.uint8)

        img_combined = np.concatenate([img, mask_class[..., None]], axis=2)

        return img_combined

In [10]:
COCO_dataset_train = COCO_Dataset('train')
COCO_dataset_val = COCO_Dataset('val')

loading annotations into memory...
Done (t=26.73s)
creating index...
index created!
loading annotations into memory...
Done (t=3.07s)
creating index...
index created!


In [11]:
len(COCO_dataset_train.img_list)

4000

Кол-во эпох и размер батча

In [12]:
NUM_EPOCHS = 4
BATCH_SIZE = 32

In [13]:
train_set = COCO_dataset_train.train_dataset(BATCH_SIZE, NUM_EPOCHS, 256)
val_set = COCO_dataset_val.val_dataset(BATCH_SIZE, 256)

#### Используем Unet на TF Keras

In [14]:
def build_model():
    x = tf.keras.layers.Input((256, 256, 3)) # входной размер картинки(для некоторых архитектура можно использовать None вместо размера)
    
    # Дальше применяем свертку, понимать это стоит так: Создаем слой и сразу вызываем его для входных данных - х
    out = tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu')(x)
    # Тут необходимо назвать по другому, чтобы результат потом пробросить в конец сети для конкатенации
    out1 = tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu')(out)
    out = tf.keras.layers.MaxPool2D((2, 2))(out1)

    out = tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu')(out)
    out2 = tf.keras.layers.Conv2D(128, (3, 3), padding='same', activation='relu')(out)
    out = tf.keras.layers.MaxPool2D((2, 2))(out2)

    out = tf.keras.layers.Conv2D(256, (3, 3), padding='same', activation='relu')(out)
    out3 = tf.keras.layers.Conv2D(256, (3, 3), padding='same', activation='relu')(out)
    out = tf.keras.layers.MaxPool2D((2, 2))(out3)

    out = tf.keras.layers.Conv2D(512, (3, 3), padding='same', activation='relu')(out)
    out4 = tf.keras.layers.Conv2D(512, (3, 3), padding='same', activation='relu')(out)
    out = tf.keras.layers.MaxPool2D((2, 2))(out4)

    out = tf.keras.layers.Conv2D(1024, (3, 3), padding='same', activation='relu')(out)
    out = tf.keras.layers.Conv2D(1024, (3, 3), padding='same', activation='relu')(out)

    # Транспонированная свертка(увеличение картинки сверткой)
    # Увеличиваем пространственные размерности страйдом (2,2)
    out = tf.keras.layers.Conv2DTranspose(512, (3, 3), strides=(2, 2), padding='same', activation='relu')(out)
    # Skip connection
    # Проброс информации о картинки пучем конкатенации, об этом говорилось ранее. Конкатенируем по 3тьему измерению(канальному)
    out = tf.concat([out4, out], axis=3)
    # Далее снова передаем в свертки
    out = tf.keras.layers.Conv2D(512, (3, 3), padding='same', activation='relu')(out)
    out = tf.keras.layers.Conv2D(512, (3, 3), padding='same', activation='relu')(out)
    # Затем опять повышаем разрешение
    out = tf.keras.layers.Conv2DTranspose(256, (3, 3), strides=(2, 2), padding='same', activation='relu')(out)
    out = tf.concat([out3, out], axis=3)

    out = tf.keras.layers.Conv2D(256, (3, 3), padding='same', activation='relu')(out)
    out = tf.keras.layers.Conv2D(256, (3, 3), padding='same', activation='relu')(out)

    out = tf.keras.layers.Conv2DTranspose(128, (3, 3), strides=(2, 2), padding='same', activation='relu')(out)
    out = tf.concat([out2, out], axis=3)

    out = tf.keras.layers.Conv2D(128, (3, 3), padding='same', activation='relu')(out)
    out = tf.keras.layers.Conv2D(128, (3, 3), padding='same', activation='relu')(out)

    out = tf.keras.layers.Conv2DTranspose(64, (3, 3), strides=(2, 2), padding='same', activation='relu')(out)
    out = tf.concat([out1, out], axis=3)

    out = tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu')(out)
    out = tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu')(out)
    # На выходе имеем 1 канал в выходной карте признаком. Тоесть каждый пиксель будет иметь вероятность принадлежности к классам фон/человек
    out = tf.keras.layers.Conv2D(1, (3, 3), padding='same', activation='sigmoid')(out)

    # Теперь просто передадим наш граф в керас
    return tf.keras.Model(inputs=x, outputs=out)

model = build_model()

In [ ]:
loss = tf.keras.losses.BinaryCrossentropy()
model.compile(optimizer='adam', loss=loss)

EXP_NAME = 'transfer'
tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir='logs/'+EXP_NAME, 
    write_graph=False, update_freq=100, profile_batch=0)

hist = model.fit(train_set, 
                 batch_size = BATCH_SIZE,
                 validation_data = val_set,
                 epochs=NUM_EPOCHS,
                 callbacks = [tensorboard_callback])

plt.plot(hist.history['loss'])

Epoch 1/4
